In [1]:
import sys

sys.path.append("../../")

from app.common.logger import logger
from app.config.utils import Configuration, init_config
from app.common.llm_clients.openai_client import OpenAILLMClient

from ml.data_processing.prompt import STEP_1_1_PREPROCESSING_PROMPT

In [2]:
config = Configuration()
init_config(config)
config = config()

# Data Load & Split paragraph

In [3]:
#  마크다운 파일 불러오기
with open("../data/amr_guide.md", "r") as f:
    amr_guide_md = f.read()

In [5]:
amr_guide_md

'# 1. Daily Operating\n\n## 1.1. System Start-up\n\n### Place Cycle - Turn ON/OFF\n\n[WEB ACS 설정 절차]\n\n1. 오른쪽 톱니바퀴 클릭\n2. Place Cycle 메뉴 선택\n3. 검색창에서 제어할 Place 이름 검색\n4. Logistics 스위치를 On으로 전환\n5. Run 상를을 RUNNING으로 전환\n\n[운영 규칙]\n\n- ON 시점: 생산 시작 전 Logistics/Run 버튼을 `ON`으로 활성화\n- OFF 시점: 생산 완료(종업) 후 `OFF`로 변경\n\n[용어 설명]\n\n- 물류 (Logistics): ON/OFF 전환\n    - 시스템이 미션을 생성하고 AMR이 이를 수신하여 수행하도록 활성화하는 기능입니다.\n    - 가동 중일 때는 반드시 `ON`, 비가동(종업) 시에는 `OFF`여야 합니다.\n- RUN: ON 활성화\n    - 가동 중인 생산 라인의 경우 필수적으로 켜야 합니다. (대부분 상시 `ON`)\n- Note: AMR 동작은 RUN과 Logistics가 모두 `ON`일 때만 가능합니다.\n\n*Reference: 5 Page of [KR]AMR Trouble Shoot_v6.1.pptx*\n\n### Place Cycle - Logistics Auto ON/Auto OFF\n\n[기능 설명]\n\n- AUTO ON/OFF: `RUN`이 활성화된 항목들에 대해 자동으로 Logistics를 ON/OFF 하는 기능입니다.\n- 확인 사항: 실제 가동 라인에서 `RUN`이 `ON` 상태인지 확인하십시오. (사용하지 않는 장소는 `RUN OFF`)\n\n[설정 경로]\n\n- (WEB ACS) 오른쪽 톱니바퀴 클릭 → Place Cycle\n\n*Reference: 5 Page of [KR]AMR Trouble Shoot_v6.1.pptx*\n\n## 1.2. System Shutdown\n\n### AMR Power Management (

In [4]:
import re

def hierarchical_split(markdown_text):
    """
    마크다운 텍스트를 # -> ## -> ### 순서로 깊이 우선 탐색하며 분할합니다.
    - # 섹션 내에 ###가 있으면 ### 단위로 분할
    - ###가 없고 ##가 있으면 ## 단위로 분할
    - 둘 다 없으면 # 단위 유지
    """
    chunks = []

    # 1단계: 가장 상위 레벨(#)로 분리
    # 헤더 텍스트(예: "# 1. Daily Operating\n")를 포함하여 분리
    level1_parts = re.split(r'(^# .+(?:\n|$))', markdown_text, flags=re.MULTILINE)

    # loop over level 1 parts
    # 0번 인덱스는 첫 헤더 전의 내용이므로 건너뜀 (필요시 처리 가능)
    for i in range(1, len(level1_parts), 2):
        h1 = level1_parts[i]       # 대제목 (Level 1 Header)
        content1 = level1_parts[i+1] # 그 아래 내용

        # 2단계: ## (Level 2) 존재 여부 확인
        if re.search(r'^## .+', content1, flags=re.MULTILINE):
            level2_parts = re.split(r'(^## .+(?:\n|$))', content1, flags=re.MULTILINE)

            # [Case] ## 헤더 나오기 전의 서문 (Intro)
            if level2_parts[0].strip():
                # 대제목 + 서문
                chunks.append(h1 + level2_parts[0])

            for j in range(1, len(level2_parts), 2):
                h2 = level2_parts[j]       # 중제목
                content2 = level2_parts[j+1]

                # 3단계: ### (Level 3) 존재 여부 확인
                if re.search(r'^### .+', content2, flags=re.MULTILINE):
                    level3_parts = re.split(r'(^### .+(?:\n|$))', content2, flags=re.MULTILINE)

                    # [Case] ### 헤더 나오기 전의 서문
                    if level3_parts[0].strip():
                        # 대제목 + 중제목 + 서문
                        chunks.append(h1 + h2 + level3_parts[0])

                    for k in range(1, len(level3_parts), 2):
                        h3 = level3_parts[k] # 소제목
                        content3 = level3_parts[k+1]

                        # [핵심] 대제목 + 중제목 + 소제목 + 내용
                        chunks.append(h1 + h2 + h3 + content3)
                else:
                    # ###가 없으면 ## 단위로 저장
                    # [핵심] 대제목 + 중제목 + 내용
                    chunks.append(h1 + h2 + content2)

        # ##는 없지만 ###가 바로 나오는 경우 (예외 케이스)
        elif re.search(r'^### .+', content1, flags=re.MULTILINE):
            level3_direct_parts = re.split(r'(^### .+(?:\n|$))', content1, flags=re.MULTILINE)

            if level3_direct_parts[0].strip():
                chunks.append(h1 + level3_direct_parts[0])

            for m in range(1, len(level3_direct_parts), 2):
                h3 = level3_direct_parts[m]
                content3 = level3_direct_parts[m+1]
                # 대제목 + 소제목 + 내용
                chunks.append(h1 + h3 + content3)

        else:
            # 하위 헤더가 전혀 없으면 # 단위로 저장
            chunks.append(h1 + content1)

    return chunks

In [22]:
paragraphs = hierarchical_split(amr_guide_md)

print(f"총 분할된 단락 수: {len(paragraphs)}")
print("-" * 30)

총 분할된 단락 수: 57
------------------------------


In [39]:
with open("../data/acs_alarm.md", "r") as f:
    acs_alarm_md = f.read()
acs_alarm_md

'| Part      | No.   | Contents (Kor)                                             | Contents (Eng)                                                          | LEVEL |\n| :-------: | :---: | :--------------------------------------------------------: | :---------------------------------------------------------------------: | :---: |\n| 테스크       | 10000 | RFID 감지 안됨                                                 | \\[10000\\] RFID not detected                                             | Warn  |\n| 테스크       | 10001 | 리프트 미션 실패                                                  | \\[10001\\] Lift mission fail                                             | Warn  |\n| 테스크       | 10002 | 턴테이블 미션 실패                                                 | \\[10002\\] Turntable mission fail                                        | Warn  |\n| 테스크       | 10003 | 도킹 중 장애물 감지                                                | \\[10003\\] Obstacle detected during docking                              | Warn

In [ ]:
def split_markdown_table(markdown_text, chunk_size=5):
    """
    마크다운 표를 파싱하여 헤더를 유지한 채로 행 단위로 분할합니다.
    """
    # 빈 줄 제거 및 줄 단위 분리
    lines = [line for line in markdown_text.strip().split('\n') if line.strip()]

    if len(lines) < 3:  # 헤더(1) + 구분선(1) + 내용(1) 최소 3줄 필요
        return [markdown_text]

    # 1. 헤더와 구분선 추출 (보통 첫 2줄)
    # | Part | No. | ... 형태라고 가정
    table_header = lines[:2]
    table_body = lines[2:]

    chunks = []

    # 2. 본문을 chunk_size만큼 순회하며 자르기
    for i in range(0, len(table_body), chunk_size):
        # 현재 청크에 들어갈 행들 추출
        batch_rows = table_body[i : i + chunk_size]

        # 헤더 + 현재 행들 결합
        chunk_lines = table_header + batch_rows

        # 다시 하나의 문자열로 합침
        chunk_text = '\n'.join(chunk_lines)
        chunks.append(chunk_text)

    return chunks

In [12]:
alarm_chunks = split_markdown_table(acs_alarm_md, chunk_size=5)
print(f"총 분할된 테이블 청크 수: {len(alarm_chunks)}")
print("-" * 50)

총 분할된 테이블 청크 수: 26
--------------------------------------------------


In [29]:
# API 문서 불러오기
with open("../data/HACS_api_doc.md", "r") as f:
    HACS_api_doc_md = f.read()

with open("../data/MCS_api_doc.md", "r") as f:
    MCS_api_doc_md = f.read()

# 에러 클리어 시나리오
with open("../data/error_scenario.md", "r") as f:
    error_scenario_md = f.read()

In [30]:
HACS_chunks = hierarchical_split(HACS_api_doc_md)
MCS_chunks = hierarchical_split(MCS_api_doc_md)
error_scenario_chunks = hierarchical_split(error_scenario_md)

In [31]:
hacs_results = []
for i, a in enumerate(HACS_chunks):
    hacs_results.append({
        "id": i,
        "preprocessed": a,
        "source": "HACS_api_doc.md"
    })
logger.info(f"hacs_results length: {len(hacs_results)}")


mcs_results = []
for i, a in enumerate(MCS_chunks):
    mcs_results.append({
        "id": i,
        "preprocessed": a,
        "source": "MCS_api_doc.md"
    })
logger.info(f"mcs_results length: {len(mcs_results)}")


error_scenario_results = []
for i, a in enumerate(error_scenario_chunks):
    error_scenario_results.append({
        "id": i,
        "preprocessed": a,
        "source": "error_scenario.md"
    })
logger.info(f"error_scenario_results length: {len(error_scenario_results)}")

2026-01-06 18:06:45.484 | INFO     | __main__:<module>:8 - hacs_results length: 76
2026-01-06 18:06:45.485 | INFO     | __main__:<module>:18 - mcs_results length: 20
2026-01-06 18:06:45.486 | INFO     | __main__:<module>:28 - error_scenario_results length: 4


In [32]:
import json

with open("../data/HACS_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(hacs_results, f, ensure_ascii=False, indent=4)

with open("../data/MCS_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(mcs_results, f, ensure_ascii=False, indent=4)

with open("../data/error_scenario_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(error_scenario_results, f, ensure_ascii=False, indent=4)

# 데이터 가공

In [15]:
model = "google/gemini-3-flash-preview"
provider = "openrouter"
api_key = config["openrouter"]["api_key"]
params = {
    "max_tokens": 2056,
    "temperature": 0.0
}
llm_client = OpenAILLMClient(model=model, provider=provider, api_key=api_key, params=params)

2026-01-02 16:13:50.598 | INFO     | app.common.llm_clients.openai_client:_create_clients:57 - Creating OpenRouter client.


In [29]:
resuls = []
for i, p in enumerate(paragraphs):
    user_prompt = f"""[Context]
    {p}"""
    response = llm_client.generate(system_prompt=STEP_1_1_PREPROCESSING_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
    result = response.choices[0].message.content
    logger.info(result)

    json_result = {
        'id': i,
        'original': p,
        'preprocessed': result,
        'source': 'amr_guide.md'
    }
    resuls.append(json_result)


2026-01-02 16:01:09.618 | INFO     | __main__:<module>:7 - WEB ACS의 Place Cycle 설정 시, AMR이 미션을 수신하고 동작하기 위해서는 Logistics 스위치와 Run 상태가 모두 ON으로 활성화되어야 한다.

WEB ACS의 Place Cycle 설정 절차는 오른쪽 톱니바퀴 아이콘 클릭 후 Place Cycle 메뉴를 선택하고, 검색창에서 제어할 Place 이름을 검색하여 Logistics 스위치와 Run 상태를 각각 ON과 RUNNING으로 전환하는 순서로 진행한다.

Place Cycle의 Logistics 기능은 시스템이 미션을 생성하고 AMR이 이를 수신하여 수행하도록 활성화하는 장치이며, 생산 가동 중에는 반드시 ON, 비가동 또는 종업 시에는 OFF 상태를 유지해야 한다.

Place Cycle의 RUN 상태는 가동 중인 생산 라인에서 필수적으로 활성화해야 하는 항목이며, 대부분의 운영 상황에서 상시 ON 상태를 유지한다.

생산 시작 전에는 Place Cycle의 Logistics와 Run 버튼을 ON으로 활성화하고, 생산 완료 후에는 해당 버튼들을 OFF로 변경하는 것을 운영 규칙으로 한다.
2026-01-02 16:01:12.185 | INFO     | __main__:<module>:7 - WEB ACS의 Place Cycle 설정 메뉴에서 AUTO ON/OFF 기능을 실행하면 RUN이 활성화된 항목에 대해 Logistics를 자동으로 ON/OFF 하며, 사용자는 실제 가동 라인의 RUN 상태가 ON인지 확인하여 사용하지 않는 장소의 RUN을 OFF로 설정해야 한다.
2026-01-02 16:01:15.197 | INFO     | __main__:<module>:7 - 1. 충전기에 도킹되어 충전 중인 AMR은 충전 상태를 유지해야 하며 절대로 전원을 끄지 않는다.

2. 충전기 외부에 위치하여 미션 대기 중인 AMR은 전원을 꺼야 하며, 종료 시 WEB ACS에서 해당 AM

In [ ]:
import json

with open("amr_guide_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(resuls, f, ensure_ascii=False, indent=4)

In [16]:
resuls2 = []
for i, a in enumerate(alarm_chunks):
    user_prompt = f"""[Context]
    {a}"""
    response = llm_client.generate(system_prompt=STEP_1_1_PREPROCESSING_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
    result = response.choices[0].message.content
    logger.info(result)

    json_result = {
        'id': i,
        'original': a,
        'preprocessed': result,
        'source': 'acs_alarm.md'
    }
    resuls2.append(json_result)

2026-01-02 16:14:17.123 | INFO     | __main__:<module>:7 - BMA 공정 AMR의 [10000] RFID 감지 안됨(RFID not detected) 경고는 주행 경로상의 RFID 태그 인식이 불가능한 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 AMR의 정지 위치를 확인하고 RFID 리더기와 태그의 정렬 상태를 점검해야 한다.

BMA 공정 AMR의 [10001] 리프트 미션 실패(Lift mission fail) 경고는 팔레트 상하강 동작이 지정된 시간 내에 완료되지 않았음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 리프트 구동부의 기계적 간섭 여부를 확인한 후 HMI를 통해 리프트 위치를 초기화해야 한다.

BMA 공정 AMR의 [10002] 턴테이블 미션 실패(Turntable mission fail) 경고는 상단 턴테이블의 회전 동작이 정상 범위 내에서 종료되지 않았을 때 발생한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 턴테이블의 회전 반경 내 이물질을 제거하고 수동 모드에서 원점 복귀를 수행해야 한다.

BMA 공정 AMR의 [10003] 도킹 중 장애물 감지(Obstacle detected during docking) 경고는 설비 진입 및 도킹 과정에서 안전 센서가 물체를 감지하여 주행이 중단된 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 도킹 경로상의 장애물을 제거한 후 주행 재개 버튼을 조작해야 한다.

BMA 공정 AMR의 [10004] 거치대 도킹 실패(Wing docking fail) 경고는 AMR이 윙(Wing) 타입 거치대와의 물리적 결합 위치 정밀도 확보에 실패했음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 거치대의 정위치 이탈 여부를 확인하고 AMR을 후진시킨 후 재도킹을 시도해야 한다

In [19]:
import json

with open("acs_alarm_preprocessed.json", "w", encoding="utf-8") as f:
    json.dump(resuls2, f, ensure_ascii=False, indent=4)